In [49]:
# WARNING: DO NOT REPEATEDLY RUN THIS CELL. DOING SO MAY HAVE US BLOCKED FROM THE WEBSITE
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import time

url = "https://www.coffeereview.com/review/?locations=na"

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
html = urlopen(req).read()

soup = BeautifulSoup(html, 'html.parser')

In [54]:
# WARNING: DO NOT REPEATEDLY RUN THIS CELL. DOING SO MAY HAVE US BLOCKED FROM THE WEBSITE
curr_review = soup.find_all(class_="review-template")[1]

review_url = curr_review.find_all('a', href=True)[3]['href']

review_req = Request(review_url, headers={'User-Agent': 'Mozilla/5.0'})
review_html = urlopen(review_req).read()

review_soup = BeautifulSoup(review_html, 'html.parser')

print("Opening URL: ", review_url)
print(review_soup)

Opening URL:  https://www.coffeereview.com/review/red-sunset-by-andres-cardona/
<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="width=device-width, initial-scale=1.0" id="dynamik-viewport" name="viewport"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>
<!-- This site is optimized with the Yoast SEO plugin v18.3 - https://yoast.com/wordpress/plugins/seo/ -->
<title>Coffee Review | Red Sunset by Andres Cardona by Nostalgia Coffee Roasters</title><link as="style" href="https://fonts.googleapis.com/css?family=Libre%20Franklin%3A300%2C300i%2C400%2C400i%2C500%2C600%2C700%2C700i&amp;display=swap" rel="preload"/><link href="https://fonts.googleapis.com/css?family=Libre%20Franklin%3A300%2C300i%2C400%2C400i%2C500%2C600%2C700%2C700i&amp;display=swap" media="print" onload="this.media='all'" rel="stylesheet"/><noscript><link href="ht